In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
#상수들
learning_rate = 0.001
batch_size = 64
epochs =3
optimizer  = torch.optim.Adam(model.parameters(),lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


In [3]:
def train(dataloader, model, loss_fn,optimizer):
    size = len(dataloader)
    model.train()
    for idx ,(X, y ) in enumerate(dataloader):
        pred = model(X)
        loss= loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if idx % 100 == 0:
            loss, current = loss.item(), idx * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size= len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss , correct = 0 , 0

    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss = loss_fn(pred,y)
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /=num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader,model,loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304253  [   64/  938]
loss: 0.558162  [ 6464/  938]
loss: 0.394680  [12864/  938]
loss: 0.514208  [19264/  938]
loss: 0.465017  [25664/  938]
loss: 0.418989  [32064/  938]
loss: 0.372771  [38464/  938]
loss: 0.522331  [44864/  938]
loss: 0.503371  [51264/  938]
loss: 0.558927  [57664/  938]
Test Error: 
 Accuracy: 84.8%, Avg loss: 0.001402 

Epoch 2
-------------------------------
loss: 0.258462  [   64/  938]
loss: 0.355386  [ 6464/  938]


KeyboardInterrupt: 

In [ ]:
import torchvision.models as models

torch.save(model.state_dict(),"model_weight.pth")